# 모델링
### **다시 모델링을 수행할 경우, 무작위 데이터 증강이 다시 수행되므로 모델 pickle 파일을 불러온 후, 테스트 단계만을 수행해야 함.**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [4]:
grid_infra = pd.read_csv('격자화_인프라.csv')
grid_infra = grid_infra[['index_right']]
grid_infra = grid_infra.set_index('index_right')
# 격자 정보는 원핫 인코딩을 통해 실수값으로 모델에 전달되지 않도록 함. => 범주화
# 격자 정보가 실수로서의 정보를 담지 않고, 격자 각각의 의미만이 있기 때문.

In [5]:
first_df = pd.read_csv('22년1분기이전_격자별.csv')
second_df = pd.read_csv('23년이전_격자별.csv')
third_df = pd.read_csv('23년전체_격자별.csv')

In [6]:
first_df.drop(1,axis=0,inplace=True)
first_df.columns = first_df.iloc[0]
first_df.drop(0,axis=0,inplace=True)
first_df.set_index('정보',inplace=True)
first_df = first_df.iloc[1:]
first_df = first_df.astype('float')

In [7]:
second_df.drop(1,axis=0,inplace=True)
second_df.columns = second_df.iloc[0]
second_df.drop(0,axis=0,inplace=True)
second_df.set_index('정보',inplace=True)
second_df = second_df.iloc[1:]
second_df = second_df.astype('float')


In [8]:
third_df.drop(1,axis=0,inplace=True)
third_df.columns = third_df.iloc[0]
third_df.drop(0,axis=0,inplace=True)
third_df.set_index('정보',inplace=True)
third_df = third_df.iloc[1:]
third_df = third_df.astype('float')


# 통합 데이터 프레임생성 및 feature engineering

- 연속형 데이터 : 각 격자 별 인프라 열
- 범주형 데이터 : 수집 일자 별 데이터 정보, 격자 정보

### feature engineering 내역
- 동일한 격자 데이터를 수집 기간으로 구분할 수 있게 수집 순서에 따른 정보를 범주값으로 구분 후 데이터셋에 추가
- 좌표정보를 모델에 포함시키기 위해 각 격자별로 범주화를 시킨 후 가변수화 수행

### Randomforest 모델 선정 이유.
- 데이터셋에 존재하는 이상치에 둔감한 모델 선정 필요.
- 수 많은 feature가 존재함에도 다중공선성, 모델 복잡성 최소화를 할 수 있는 모델 선정 필요.
- 데이터 증강에 따른 과적합을 하이퍼 파라미터 튜닝으로 최소화 할 수 있는 모델 선정 필요.

In [9]:
first_df['수집순서'] = '1_수집순서'
second_df['수집순서'] = '2_수집순서'
third_df['수집순서'] = '3_수집순서'

In [10]:
first_df = first_df.reset_index()
second_df = second_df.reset_index()
third_df = third_df.reset_index()
# 이미 정보가 object 형으로 전달되어 있기 때문에 별도의 속성 변경은 수행하지 않음.

In [11]:
first_df = pd.get_dummies(first_df,drop_first=False)
second_df = pd.get_dummies(second_df,drop_first=False)
third_df = pd.get_dummies(third_df,drop_first=False)
# 수집 순서 및 격자의 범주화

In [12]:
meta = pd.concat([first_df,second_df])
#통합 데이터 프레임 선언

In [13]:
meta = meta.fillna(0)
third_df = third_df.fillna(0)

# concat에 따른 결측치 값 0으로 처리

In [14]:
meta = meta.drop('정보_114286.0',axis=1)
meta['수집순서_3_수집순서'] = 0
# 예측 데이터셋과의 통일성을 위해 특정 열 제거 및 추가

In [15]:
third_df[['수집순서_1_수집순서','수집순서_2_수집순서']] = 0
# 학습 데이터셋과의 통일성을 위해 특정 열 추가

---

In [16]:
meta.to_csv('종합학습데이터프레임.csv',index=False)

---

# 데이터 증강
- 학습 데이터로 활용하기 위한 행의 개수가 부족함 => 주어진 데이터에 노이즈를 추가해, 데이터 증강 시행
- 데이터 증강으로 인한 과적합 우려 => 복원 추출을 최소화하고 연속형 데이터에만 증강을 수행함. 증강한 데이터에는 노이즈를 추가함.
- 범주형 데이터에는 데이터 증강을 수행하지 않음. => 연속 데이터가 아니기에 노이즈가 의미가 없음.

In [17]:
for_arg_col = meta.columns[:31]
# 연속형 데이터 열

In [18]:
for_arg_col = for_arg_col.drop('단속장소')

In [19]:
# 데이터셋 로딩 및 초기 탐색

data = meta.copy()

# 결측치 확인 및 데이터셋 크기
missing_values = data.isnull().sum()
dataset_size = data.shape


# 특성 표준화
from sklearn.preprocessing import StandardScaler

X = data.drop('단속장소', axis=1)  # 특성
y = data['단속장소']              # 타겟

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[for_arg_col])
# 연속형 데이터셋에 대해서만 표준화
X[for_arg_col] = X_scaled
# 범주형 데이터와 통합

In [20]:
def add_gaussian_noise(X, noise_level=0.02):
    noise = np.random.normal(0, noise_level, X.shape)
    return X + noise

X_augmented = add_gaussian_noise(X_scaled)
# 연속형 데이터셋에만 데이터 변형
X_full_augmented = X.copy()
# 변형된 데이터셋의 범주형 데이터에는 변형이 되지 않음.
X_full_augmented[for_arg_col] = X_augmented
# 변형된 데이터셋의 연속형, 범주형 데이터 통합

X_augmented_2 = add_gaussian_noise(X_scaled)
# 연속형 데이터셋에만 데이터 변형
X_full_augmented_2 = X.copy()
# 변형된 데이터셋의 범주형 데이터에는 변형이 되지 않음.
X_full_augmented_2[for_arg_col] = X_augmented_2
# 변형된 데이터셋의 연속형, 범주형 데이터 통합

In [21]:
X_full_augmented = pd.concat([X_full_augmented,X_full_augmented_2])

In [22]:
# 데이터 증강
X_combined = pd.concat([X,X_full_augmented])
y_combined = np.concatenate((y, y,y))
# 표준편차를 이용한 노이즈값을 데이터셋에  추가
# 균일한 값의 범위를 위해 표준화된 데이터셋에 노이즈를 추가.


In [23]:
# 모델 훈련 및 평가 - 선형 회귀
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42)

# 교차검증

In [24]:
from sklearn.ensemble import RandomForestRegressor

In [25]:

cv_arr_r2 = []
cv_arr_rmse = []
cv_arr_mse = []
cv_arr_mape = []

best_params = {
    'n_estimators' : 150,
    'max_features' : 'sqrt',
    'max_depth' : 15,
    'criterion' : 'squared_error',
    'bootstrap' : True
}
for i in range(10):
    rf_cv_model=RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_features=best_params['max_features'],
    max_depth=best_params['max_depth'],
    criterion=best_params['criterion'],
    )
    # 무작위성을 위해 랜덤 시드 삭제
    rf_cv_model.fit(X_train, y_train)
    y_cv_pred_rf = rf_cv_model.predict(X_test)
    cv_arr_rmse.append(mean_absolute_error(y_test, y_cv_pred_rf)**0.5)
    cv_arr_r2.append(r2_score(y_test, y_cv_pred_rf))
    cv_arr_mse.append(mean_absolute_error(y_test, y_cv_pred_rf))
    cv_arr_mape.append(mean_absolute_percentage_error(y_test, y_cv_pred_rf))
    
    #성능 안정성 교차검증

In [26]:
cv_arr_r2

[0.8925888017790399,
 0.8959166580051174,
 0.8868035961272791,
 0.8870742303195774,
 0.8939890371231848,
 0.8914373139504564,
 0.8981556637949407,
 0.8811086772923128,
 0.8920367714164612,
 0.887609182371542]

In [27]:
cv_arr_mse

[413.15923809523804,
 405.26219150858174,
 423.1064308943089,
 428.3360975609756,
 429.0296260162602,
 402.21407317073175,
 405.00248238482396,
 434.98372899729,
 417.5832899728997,
 436.0328048780487]

**최적의 하이퍼 파라미터 적용**

In [28]:

rf_model = RandomForestRegressor( 
    n_estimators=best_params['n_estimators'],
    max_features=best_params['max_features'],
    max_depth=best_params['max_depth'],
    criterion=best_params['criterion'],
    random_state=42
    )
rf_model.random_state = 42
# 최적의 성능을 내는 하이퍼 파라미터 적용 및 랜덤 시드 값 적용
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)


In [29]:
# 테스트 데이터셋에 대한 예측
test_data = third_df.copy()

X_test_new_corrected = test_data.drop('단속장소', axis=1)
X_test_new_standardized = scaler.transform(X_test_new_corrected[for_arg_col])
X_test_new_corrected[for_arg_col] = X_test_new_standardized
X_test_new_corrected.columns = X_train.columns
X_test_new_corrected['수집순서_3_수집순서'] = 1
X_test_new_corrected['수집순서_1_수집순서'] = 0

y_pred_test_corrected = rf_model.predict(X_test_new_corrected)

# 테스트 데이터셋 평가
y_actual_test = test_data['단속장소']
mse_test = mean_squared_error(y_actual_test, y_pred_test_corrected)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_actual_test, y_pred_test_corrected)

In [30]:
r2_rf
# 학습 데이터에 대한 예측 성능 (r2값)

0.8948859184019244

In [31]:
rmse_test

606.4527733882367

In [32]:
r2_test
# 테스트 데이터에 대한 예측 성능 (r2값)

0.8298342981099636

- 테스트 데이터와 학습 데이터 간 예측 성능에 큰 차이가 없으므로 과적합 우려가 낮다고 할 수 있음.
- 학습 데이터에 대한 예측 성능에 큰 차이를 보이지 않으므로 과적합 우려가 낮다고 할 수 있음.

In [33]:
# ## 모델 저장하기
# ## 사용하기 위해 주석 처리 해제 필요
# import pickle

# with open('model.pkl', 'wb') as file:
#         pickle.dump(rf_model, file)

----

# 모델 불러오기
- 데이터 증강은 무작위로 이뤄지기 때문에 이후 다시 학습을 진행할 경우, 계산할 오차를 정확하게 측정하지 못하게 됨.


In [34]:
import pickle
with open('model.pkl', 'rb') as file:
    rf_model = pickle.load(file)
# 모델 불러오기

In [35]:
rf_model

RandomForestRegressor(max_depth=15, max_features='sqrt', n_estimators=150,
                      random_state=42)

# 테스트
## 1. 변수 설명
- X_test_new_corrected : 원본 테스트 데이터셋 (target 제거한 상태)
- X_test_cctv_modified : CCTV 개수 데이터 변화 대상 데이터셋
- X_test_cctv_modified_standardized : CCTV 개수 변화 데이터셋 (표준화한 상태)
- y_pred_cctv_modified : 모델의 예측 Y값 (X_test_cctv_modified_standardized을 대상으로 예측)
- y_pred_cctv_not_modified : 모델의 예측 Y값 (X_test_new_standardized  대상으로 예측)
- X_test_new_standardized : 원본 테스트 데이터셋 (표준화한 상태)
- y_pred_current : 실제 데이터 셋의 Y값
- predicted_change : 실제 데이터 기반 예측값 과 변화 데이터 기반 예측값의 차이

- 실제 단속 건수와 변화된 데이터셋의 예측값과 비교함으로써 변화하는 단속건수 확인

In [36]:
# 'cctv위치' 값이 증가할 때 각 격자 위치에서의 불법 주정차 적발량 변화 예측

# 현재 CCTV 위치 정보 복사
X_test_cctv_modified = test_data.copy()
X_test_cctv_modified = X_test_cctv_modified.drop('단속장소',axis=1)
X_test_cctv_modified.columns = X_train.columns
X_test_cctv_modified['수집순서_3_수집순서'] = 1
X_test_cctv_modified['수집순서_1_수집순서'] = 0

plus_cctv_value = 1 # 증가시킬 cctv 값

# CCTV 위치를 1 증가시킴 (가정: 현재 값 + 1)
X_test_cctv_modified['cctv위치'] = X_test_cctv_modified['cctv위치'] + plus_cctv_value

# 변경된 데이터 표준화
X_test_cctv_modified_standardized = scaler.transform(X_test_cctv_modified[for_arg_col])
X_test_cctv_modified[for_arg_col] = X_test_cctv_modified_standardized




In [37]:
# 변경된 데이터에 대한 예측 수행
y_pred_cctv_modified = rf_model.predict(X_test_cctv_modified)
y_pred_cctv_not_modified = rf_model.predict(X_test_new_corrected)
# 예측된 적발량 변화 계산 (CCTV 추가 시 예측 적발량 - 현재 예측 적발량)
predicted_change = y_pred_cctv_modified - y_actual_test


In [38]:
result = pd.DataFrame()
result['실제_카메라수'] = test_data['cctv위치']
# 실제 설치된 격자 별 CCTV 카메라 수를 의미.
result['더해진_카메라수'] = test_data['cctv위치'] + plus_cctv_value
# 변경된 CCTV 카메라 수를 의미.
result['실제_단속건수'] = y_actual_test
# 실제 데이터의 단속건수를 의미.
result['예측_단속건수(변화)'] = y_pred_cctv_modified
# 변경 데이터를 기반으로 예측한 단속건수를 의미.
result['예측_단속건수(원본)'] = y_pred_cctv_not_modified
# 원본 데이터를 기반으로 예측한 단속건수를 의미.

result['오차율(%)'] = (((y_actual_test -  y_pred_cctv_not_modified)).abs() / y_actual_test).abs()*100
# (원본 데이터셋  실제값  - 원본 데이터셋 예측값 ).절댓값/ 원본 데이터셋  실제값
result['단속건수_변화량'] = predicted_change.tolist()
# 변화된 데이터셋 예측값 - 원본 데이터셋  실제값
result['단속건수_변화량(예측)'] = y_pred_cctv_modified - y_pred_cctv_not_modified
# 변화된 데이터셋 예측값 - 원본 데이터셋 예측값

- 오차율을 계산함으로써 예측 정확도가 높은 행의 변화량을 집중해서 관찰할 수 있게 함
- 낮은 오차율의 기준을 정하기 위해 오차율의 분포를 구하고 그 분포의 중앙값보다 작은 오차율을 가진 행을 선별함.
- 단속건수_변화량(예측) 열을 기준으로 내림차순 해, CCTV 추가 설치 격자를 선정함.
- 추가 설치 대수는 5대 이므로 내림차순 기준 상위 5개 행의 격자를 선택

In [39]:
er_md = result['오차율(%)'].median()

In [40]:
er_md

29.78470798284733

In [41]:
result = result[result['오차율(%)']<er_md].sort_values('단속건수_변화량(예측)',ascending=False)
# 오차율의 중앙값의 이하 행을 선택하고 단속건수 변화량(예측)을 기준으로 내림차순

In [42]:
top_5 =  result.head()
#상위 5개 데이터프레임
top_5

,실제_카메라수,더해진_카메라수,실제_단속건수,예측_단속건수(변화),예측_단속건수(원본),오차율(%),단속건수_변화량,단속건수_변화량(예측)
1,1.0,2.0,261.0,229.331333,187.111333,28.309834,-31.668667,42.220000
0,0.0,1.0,170.0,160.626667,141.340000,16.858824,-9.373333,19.286667
2,2.0,3.0,3836.0,4461.600000,4445.520000,15.889468,625.600000,16.080000
8,4.0,5.0,564.0,600.213333,584.820000,3.691489,36.213333,15.393333
4,3.0,4.0,1684.0,2184.706667,2175.560000,29.190024,500.706667,9.146667


In [43]:
find_grid = pd.read_csv('23년전체_격자별.csv')

find_grid.drop(1,axis=0,inplace=True)
find_grid.columns = find_grid.iloc[0]
find_grid.drop(0,axis=0,inplace=True)
find_grid.set_index('정보',inplace=True)
find_grid = find_grid.iloc[1:]
find_grid = find_grid.astype('float')

In [44]:
find_grid = find_grid.iloc[list(top_5.index)]
# 대상 5개 격자의 index를 활용해, 추출


---

In [45]:
find_grid_dict = {'grid_cand' : list(find_grid.index)}
with open('grid_candidate.pkl','wb') as file:
    pickle.dump(find_grid_dict,file)
# 대상 후보 격자 파일 저장